环境用的：test

https://www.bilibili.com/video/BV1pfKNe8E7F/?spm_id_from=333.1007.tianma.3-4-10.click&vd_source=a0d30acd76c4a4ca61e1f938925ede3e

In [1]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install transformer
# !pip install datasets
# !pip install peft
# !pip install bitsandbytes

In [3]:
# 测试模型是否可用
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 运行通过了
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

print("----模型加载成功----")

/home/ubuntu/anaconda3/envs/test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


----模型加载成功----


In [4]:
# 制作数据集
from data_prepare import samples
import json

with open("datasets.jsonl","w",encoding="utf-8") as f:
    for s in samples:
        json_line = json.dumps(s) # 将每行数据转换为json格式
        f.write(json_line + "\n") # 写入文件
    else:
        print("----数据准备完成----")

----数据准备完成----


In [5]:
# 准备训练集和测试集
from datasets import load_dataset

dataset = load_dataset("json",data_files={"train":"datasets.jsonl"},split="train")
print("数据量为：",len(dataset))

print("----数据集准备完成----")

# 分割数据集
train = dataset.train_test_split(test_size=0.2)

train_dataset = train["train"]
test_dataset = train["test"]

print("训练集数据量为：",len(train_dataset))
print("测试集数据量为：",len(test_dataset))

print("----数据集分割完成----")

Generating train split: 50 examples [00:00, 12659.37 examples/s]

数据量为： 50
----数据集准备完成----
训练集数据量为： 40
测试集数据量为： 10
----数据集分割完成----


In [6]:
# 编写tokenizer的处理工具
def tokenizer_function(examples):
    # 将 examples 字典中的 prompt 列表和 completion 列表中的对应元素配对。
    # 将每对 prompt 和 completion 拼接成一个字符串，并在它们之间插入一个换行符。
    # 将所有这些拼接后的字符串放入一个新的列表 texts 中。
    texts  = [f"{prompt}\n{completion}" for prompt, completion in zip(examples["prompt"],examples["completion"])]
    print(texts)

    tokens = tokenizer(texts, truncation=True, max_length=512, padding="max_length")
    # 将 tokens 字典中的 input_ids 列表复制到 label 列表中。
    tokens["label"] = tokens["input_ids"].copy() 

    return tokens

# batch 参数表示是否对数据进行批处理
tokenized_train_dataset = train_dataset.map(tokenizer_function,batched=True)
tokenized_test_dataset = test_dataset.map(tokenizer_function,batched=True)

print("----数据集tokenize完成----")

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

['Question 20: How do you develop confidence in your singing voice?\nAnswer 20: Record and listen to your progress, focus on your strengths, and perform regularly to build confidence over time.', 'Question 8: How do you improve vocal tone quality?\nAnswer 8: Focus on resonance and placement, and practice vocal exercises like sirens to achieve a smoother tone.', 'Question 37: How can you improve your rhythm as a singer?\nAnswer 37: Practice with a metronome, clap rhythms, and sing along with instrumental tracks to improve timing and precision.', "Question 38: What are the challenges of singing in a group or choir?\nAnswer 38: Challenges include blending with others, maintaining pitch, and following the conductor's cues for dynamics and timing.", 'Question 48: What is the importance of storytelling in singing?\nAnswer 48: Storytelling connects you with the audience, making your performance more impactful and memorable.', 'Question 32: What are the benefits of recording your singing pract

Map: 100%|██████████| 10/10 [00:00<00:00, 518.55 examples/s]

['Question 18: How do you transition smoothly between vocal registers?\nAnswer 18: Practice blending your chest and head voice through exercises like sirens and scales to achieve a seamless transition.', 'Question 9: What techniques help develop vibrato in singing?\nAnswer 9: Practice sustaining notes while gently oscillating your pitch; focus on relaxation and consistent airflow.', 'Question 43: How do you recover from vocal strain or overuse?\nAnswer 43: Rest your voice, stay hydrated, and avoid whispering, which can further strain your vocal cords.', 'Question 6: How can you protect your vocal cords from strain?\nAnswer 6: Avoid over-singing, stay hydrated, and limit exposure to irritants like smoke or excessively dry air.', 'Question 47: How can you expand your vocal range safely?\nAnswer 47: Gradually practice exercises that stretch your range, like sirens, without pushing or straining your voice.', 'Question 25: How do you develop a unique singing style?\nAnswer 25: Experiment wi

In [7]:
# 量化设置
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,   # 加载8位模型
)

model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=quantization_config, device_map="auto")

print("----模型量化完成----")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


----模型量化完成----


In [8]:
# Lora 微调设置
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,  # Lora 矩阵的秩（rank）。秩决定了 Lora 矩阵的维度，通常是一个较小的整数。较小的秩可以减少计算量，但可能会影响模型的性能。
    lora_alpha=16,  # Lora 矩阵的缩放因子。这个参数用于调整 Lora 矩阵的权重，通常是一个较大的整数。较大的值可以增强 Lora 矩阵的影响。
    lora_dropout=0.05,  # Lora 矩阵的 dropout 概率。dropout 是一种正则化技术，用于防止过拟合。这个参数指定了在训练过程中随机丢弃 Lora 矩阵中某些元素的概率。
    task_type=TaskType.CAUSAL_LM,  # 任务类型。`TaskType.CAUSAL_LM` 表示这是一个因果语言建模任务，即预测下一个词的任务。
)

model = get_peft_model(model, lora_config)
print("------------------------")
model.print_trainable_parameters()

print("----Lora 微调设置完成----")

------------------------
trainable params: 1,089,536 || all params: 1,778,177,536 || trainable%: 0.0613
----Lora 微调设置完成----


In [ ]:
# 设置训练参数
from transformers import TrainingArguments,Trainer

trainning_args = TrainingArguments(
    output_dir="./main_results",
    num_train_epochs=100, 
    per_device_train_batch_size=4, # 每个GPU的batch size
    gradient_accumulation_steps=8, # 梯度累积步数
    fp16=True,
    logging_steps=10, # 每多少步保存一次日志
    save_steps=10, # 每多少步保存一次模型
    eval_strategy='steps', # 评估策略，steps表示每多少步评估一次
    eval_steps=10, # 每多少步评估一次
    learning_rate=5e-5,
    logging_dir='./main_logs',
    run_name = "Deepseek_main_finetune" # 运行名称
)

print("----训练参数设置完成----")

----训练参数设置完成----


In [10]:
# 定义训练器
trainer = Trainer(
    model=model,
    args=trainning_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

print("----训练器定义完成----")

trainer.train()

print("----训练完成----")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


----训练器定义完成----


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.16 GiB. GPU 1 has a total capacity of 10.00 GiB of which 0 bytes is free. Process 18536 has 17179869184.00 GiB memory in use. Process 18536 has 17179869184.00 GiB memory in use. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 8.42 GiB is allocated by PyTorch, and 26.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)